In [ ]:
import snowflake.connector
import pandas as pd

In [ ]:


def run_query_(query, snowflakeaccount, snowflakeuser, snowflakepassword):
    with snowflake.connector.connect(account=snowflakeaccount,
                                   user=snowflakeuser,
                                   password=snowflakepassword,
                                   warehouse='COMPUTE_WH',
                                    disable_ocsp_checks=True) as conn:
        with conn.cursor() as cur:
            results = cur.execute(query).fetchall()

    return results

In [ ]:
database = "POLICYDB"
query = f"""            select  PRIVILEGE,
                    GRANTED_ON,
                    "NAME",
                    TABLE_CATALOG,
                    TABLE_SCHEMA,
                    GRANTED_TO,
                    GRANTEE_NAME,
            from    SNOWFLAKE.ACCOUNT_USAGE.GRANTS_TO_ROLES
            where   GRANTED_ON IN ('SCHEMA', 'TABLE', 'DATABASE') and
                    PRIVILEGE in ('SELECT', 'USAGE', 'OWNERSHIP') and
                    TABLE_CATALOG = '{database}' and
                    DELETED_ON is null"""

columns =   ["PRIVILEGE",
            "GRANTED_ON",
            "NAME",
            "TABLE_CATALOG",
            "TABLE_SCHEMA",
            "GRANTED_TO",
            "GRANTEE_NAME"]
results = run_query_(query, account, user, password)
df = pd.DataFrame(results, columns=columns)
tables = df[df['GRANTED_ON'] == 'TABLE']
valid_grants = list()

for row in tables.itertuples():
    table_name = row.NAME
    table_schema = row.TABLE_SCHEMA
    table_catalog = row.TABLE_CATALOG
    role = row.GRANTEE_NAME
    privilege = row.PRIVILEGE
    print(f"Table: {table_name}, Schema: {table_schema}, Catalog: {table_catalog}, Role: {role} Is User: {row.GRANTED_TO == 'USER'}")
    schema_usage = df[(df['NAME'] == table_schema) & (df['TABLE_CATALOG'] == table_catalog) & (df["GRANTEE_NAME"] == role)]    
    schema_usage = schema_usage["PRIVILEGE"].to_list()
    database_usage = df[(df['NAME'] == table_catalog)  & (df["GRANTEE_NAME"] == role)]
    database_usage = database_usage["PRIVILEGE"].to_list()

    if privilege in ["SELECT", "OWNERSHIP"] and \
        ("USAGE" in schema_usage or "OWNERSHIP" in schema_usage) and \
              ("USAGE" in database_usage or "OWNERSHIP" in database_usage):
        grant = {"catalog": table_catalog,
                 "schema": table_schema,
                 "table": table_name,
                 "role_user": role,
                 "is_user": row.GRANTED_TO == "USER"}
        valid_grants.append(grant)

valid_grants = pd.DataFrame(valid_grants)
query = """with
   -- CTE gets all the roles each role is granted
   ROLE_MEMBERSHIPS(ROLE_GRANTEE, ROLE_GRANTED_THROUGH_ROLE)
   as
    (
    select   GRANTEE_NAME, "NAME"
    from     SNOWFLAKE.ACCOUNT_USAGE.GRANTS_TO_ROLES
    where    GRANTED_TO = 'ROLE' and
             GRANTED_ON = 'ROLE' and
             DELETED_ON is null
    ),
    -- CTE gets all roles a user is granted
    USER_MEMBERSHIPS(ROLE_GRANTED_TO_USER, USER_GRANTEE, GRANTED_BY)
    as
     (
     select ROLE,
            GRANTEE_NAME,
            GRANTED_BY
     from SNOWFLAKE.ACCOUNT_USAGE.GRANTS_TO_USERS
     where DELETED_ON is null
     )
-- 
select
        USER_GRANTEE,
        case
            when ROLE_GRANTED_THROUGH_ROLE is null
                then ROLE_GRANTED_TO_USER 
            else ROLE_GRANTED_THROUGH_ROLE
        end
        EFFECTIVE_ROLE,
from    USER_MEMBERSHIPS U
    left join ROLE_MEMBERSHIPS R
        on U.ROLE_GRANTED_TO_USER = R.ROLE_GRANTEE
;
"""
results = run_query_(query, account, user, password)
columns = ["USER_GRANTEE", "EFFECTIVE_ROLE"]
user_roles = pd.DataFrame(results, columns=columns)
query = """    select 
            GRANTEE_NAME,
             ROLE,
     from SNOWFLAKE.ACCOUNT_USAGE.GRANTS_TO_USERS
     where DELETED_ON is null"""
results = run_query_(query, account, user, password)
columns = ["GRANTEE_NAME", "ROLE"]
user_roles2 = pd.DataFrame(results, columns=columns)
user_roles2.rename(columns={"GRANTEE_NAME": "USER_GRANTEE", "ROLE": "EFFECTIVE_ROLE"}, inplace=True)
combined_user_roles = pd.concat([user_roles, user_roles2], ignore_index=True)
combined_user_roles.drop_duplicates(inplace=True)
roles = valid_grants['role_user'].unique()
policies = list()

for role in list(roles):
    print(f"Checking role: {role}")
    valid_grants_per_role = valid_grants[valid_grants['role_user'] == role]

    tables = list()
    for row in valid_grants_per_role.itertuples():
        table_name = row.table
        table_schema = row.schema
        table_catalog = row.catalog
        role = row.role_user
        print(f"Table: {table_name}, Schema: {table_schema}, Catalog: {table_catalog}, Role: {role} Is User: {row.is_user}")
    users_for_this_role = combined_user_roles[combined_user_roles['EFFECTIVE_ROLE'] == role]
    if not users_for_this_role.empty:
        print(f"Role {role} is effective for user {users_for_this_role['USER_GRANTEE'].values}")
        tables.append({
            "table": table_name,
            "schema": table_schema,
            "catalog": table_catalog,
        })
    if tables:
        policy = {
            "role": role,
            "tables": tables,
            "is_user": row.is_user,
            "users": users_for_this_role['USER_GRANTEE'].values.tolist()
        }
        policies.append(policy)


In [ ]:
policies